In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load data
sample_info = pd.read_csv('data/brain_sample_descriptions_PFC.csv')
gene_data = pd.read_csv('data/braindat.csv', index_col=0).T

# Merge datasets on common identifier
data = sample_info.merge(gene_data, left_on='!Sample_geo_accession', right_index=True, how='inner')

# Extract features and target variable
X = data.drop(columns=['Disease'])
y = data['Disease']  # Assuming binary target variable

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features for logistic regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply PCA for logistic regression with Lasso
pca = PCA(n_components=0.95)  # Retain 95% of variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Logistic Regression with Lasso
lasso_log_reg = LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=42)
lasso_log_reg.fit(X_train_pca, y_train)
y_pred_lasso = lasso_log_reg.predict(X_test_pca)
lasso_accuracy = accuracy_score(y_test, y_pred_lasso)

# Decision Tree Classifier
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)
y_pred_tree = decision_tree.predict(X_test)
tree_accuracy = accuracy_score(y_test, y_pred_tree)

# Report accuracies
print(f"Logistic Regression with Lasso (using PCA): Accuracy = {lasso_accuracy:.4f}")
print(f"Decision Tree Classifier: Accuracy = {tree_accuracy:.4f}")
